# Data Wrangling
This notebook is intended to fetch the data from the planet API for use in the project

###### Imports and set up

In [1]:
import os
import requests as req
import json
import numpy as np
import pandas as pd

In [2]:
from explore import PlanetAPI

In [3]:
file_dir = os.getcwd()
with open(os.path.join(file_dir, 'AccessCodes', 'data.txt'), 'r') as f:
    API_Keys = json.load(f)

In [4]:
oAPI = PlanetAPI(API_Keys['planet'])

###### Define
* test_for_coverage :: determine if there are any images that meet the specified criteria
* convert_fetched_data :: extract data from the json returned object
* 

In [5]:
def test_for_coverage():
    df = pd.DataFrame.from_dict(oAPI.query_stats().json()['buckets'])
    df = df[df['count'] > 0]
    df.reset_index(inplace=True, drop=True)
    return df.count()['count'] > 0

In [6]:
def convert_fetched_data(data):
    feature_data = pd.DataFrame.from_dict(data['features'])
    image_data = pd.concat(
        [feature_data.drop(['_links', 'geometry', 'properties', '_permissions'], axis=1),
         feature_data['properties'].apply(pd.Series),
         feature_data['geometry'].apply(pd.Series),
         feature_data['_links'].apply(pd.Series).rename({'_self':'self'}, axis=1)
        ],
        axis=1,
        keys=[
            'info',
            'properties',
            'geometry',
            'links'
        ]
    )
    return image_data

###### Inspection to see if the area of interest we have defined has images to pull

In [8]:
oAPI.set_geo_area_of_interest(
    (-122.53601074218751, 37.71234379341038),
    (-122.35404968261719, 37.80788523279169)
)

In [9]:
df = pd.DataFrame.from_dict(oAPI.query_stats().json()['buckets'])
df = df[df['count'] > 0]
df.reset_index(inplace=True, drop=True)

In [10]:
df

,count,start_time
0,4,2016-07-06T00:00:00.000000Z
1,3,2016-07-11T00:00:00.000000Z
2,4,2016-07-14T00:00:00.000000Z
3,1,2016-07-17T00:00:00.000000Z
4,3,2016-07-27T00:00:00.000000Z
5,2,2016-07-31T00:00:00.000000Z


###### Fetch information about the images to retreive

In [11]:
data = oAPI.query_image_metadata().json()
df_img = convert_fetched_data(data)

In [12]:
i = np.random.randint(df_img.count()[1])

df_img['properties']['item_type'].loc[i], df_img['info']['id'].loc[i]

('REOrthoTile', '20160714_193446_1056517_RapidEye-4')

In [13]:
with req.Session() as s:
    item = s.get(("https://api.planet.com/data/v1/item-types/{}/items/{}/assets/").format(
        df_img['properties']['item_type'].loc[i],
        df_img['info']['id'].loc[i]),
        auth=(API_Keys['planet'],'')
    )
    
    url = item.json()['visual']["_links"]["activate"]
    
    response = s.post(url, auth=(API_Keys['planet'],''))

response.json()

{'general': [{'message': 'Download quota has been exceeded.'}], 'field': {}}

In [14]:
with req.Session() as s:
    response = s.get(url, auth=(API_Keys['planet'],''))
    
response.json()

{'general': [{'message': 'Download quota has been exceeded.'}], 'field': {}}